In [1]:
import json
import requests
import partridge as ptg
import pandas as pd
import datetime

In [2]:
inpath = "tisseo_gtfs.zip"
service_ids_by_date = ptg.read_service_ids_by_date(inpath)
service_ids=service_ids_by_date[datetime.date(2020, 2, 3)]
view = {
    'trips.txt': {'service_id': service_ids},
}

fd=ptg.load_feed(inpath, view)
stops=fd.stops
stops_id=list(stops['stop_id'])
print("num of stop = ",len(stops_id))
print(stops_id)

num of stop =  3653
['3377699720880573', '3377699720880578', '3377699720880579', '3377699720880580', '3377699720880581', '3377699720880582', '3377699720880593', '3377699720880594', '3377699720880595', '3377699720880596', '3377699720880597', '3377699720880598', '3377699720880634', '3377699720880648', '3377699720880649', '3377699720880657', '3377699720880658', '3377699720880659', '3377699720880668', '3377699720880669', '3377699720880670', '3377699720880671', '3377699720880672', '3377699720880674', '3377699720880676', '3377699720880678', '3377699720880679', '3377699720880680', '3377699720880710', '3377699720880711', '3377699720880713', '3377699720880719', '3377699720880720', '3377699720880734', '3377699720880735', '3377699720880752', '3377699720880757', '3377699720880758', '3377699720880759', '3377699720880761', '3377699720880799', '3377699720880801', '3377699720880802', '3377699720880805', '3377699720880806', '3377699720880808', '3377699720880809', '3377699720880810', '3377699720880811',

In [3]:
stops=fd.stops
stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
0,3377699720880573,2211,Ecole Croix-Rouge,43.6052,1.38708,0,1970324837184593,2
1,3377699720880578,4531,Matabiau Gare SNCF,43.6109,1.45271,0,1970324840530381,1
2,3377699720880579,40130,Marengo-SNCF,43.6101,1.45562,0,1970324837184594,1
3,3377699720880580,40131,Marengo-SNCF,43.6102,1.45553,0,1970324837184594,1
4,3377699720880581,4481,Marengo-SNCF,43.6108,1.45539,0,1970324837184594,1
...,...,...,...,...,...,...,...,...
3648,3377704015497108,73000,Joulin,43.5503,1.42084,0,1970329131942546,1
3649,3377704015497109,73001,Joulin,43.5506,1.42122,0,1970329131942546,1
3650,3377704015497112,15045,Peries,43.5133,1.50095,0,1970324837186501,1
3651,3377704015497113,20580,Ancien Tri Postal,43.5842,1.38652,0,1970329131942548,1


In [4]:
def getdata(stop_id):
    api_token = "e7f613b8-a33f-46c0-bc6d-d143bbcfad16"
    head = {'Authorization': api_token}
    api_url_base="https://api.tisseo.fr/v1/stops_schedules.json?stopPointId="+stop_id+"&datetime=2020-02-03 05:00&number=100"
    response = requests.get(api_url_base,headers=head)
    print(response.status_code)
    file=response.json()
    print("result = ",len(file["departures"]["departure"]))
    return file

In [5]:
def to_seconds(time):
    #time  : '2020-02-03 06:06:00'
    tiemdelta=datetime.datetime.strptime(time,"%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime('2020-02-03 00:00:00',"%Y-%m-%d %H:%M:%S")
    seconds = tiemdelta.total_seconds()
    return seconds

In [6]:
def to_date(sec):
    date = datetime.timedelta(seconds=sec)+datetime.datetime.strptime('2020-02-03 00:00:00',"%Y-%m-%d %H:%M:%S")
    return date

In [7]:
def calcul_retard(L1,L2,interval1=120,interval2=600):
    #L1 : planning
    #L2 : real time
    len1 = len(L1)
    len2 = len(L2)
    i=0
    j=0
    L3=[]
    #positif lat ,negatif ealier
    while(i<len1 and j <len2):
        if abs(L2[j]-L1[i])<=interval1:
            L3.append((to_date(L2[j]),L2[j]-L1[i]))
            j+=1
            i+=1
        elif L1[i]-L2[j]>=interval2:
            j+=1
        else:
            i+=1
    return L3

In [8]:
def get_retard(stops_id):
    num_stop=0
    result={}
    for stop in stops_id:
        
        num_stop+=1
        print("stop :", stop ," is treated ",num_stop/len(stops_id)*100,"% completed")
        dataset=getdata(stop)
        
        time_series=[i['dateTime'] for i in dataset['departures']['departure']]
        time_series = list(map(to_seconds,time_series))
        
        df=fd.stop_times[fd.stop_times["stop_id"]==stop]
        planning=list(df.sort_values(by='arrival_time')["arrival_time"])
        
        retard = calcul_retard(planning,time_series)
        result[stop] = retard
        
    return result

In [9]:
#stops_id=['3377699720880573', '3377699720880578']

In [10]:
result=get_retard(stops_id)
result

stop : 3377699720880573  is treated  0.027374760470845878 % completed
200
result =  100
stop : 3377699720880578  is treated  0.054749520941691755 % completed
200
result =  100
stop : 3377699720880579  is treated  0.08212428141253764 % completed
200
result =  100
stop : 3377699720880580  is treated  0.10949904188338351 % completed
200
result =  100
stop : 3377699720880581  is treated  0.13687380235422938 % completed
200
result =  100
stop : 3377699720880582  is treated  0.16424856282507527 % completed
200
result =  100
stop : 3377699720880593  is treated  0.19162332329592116 % completed
200
result =  100
stop : 3377699720880594  is treated  0.21899808376676702 % completed
200
result =  100
stop : 3377699720880595  is treated  0.24637284423761294 % completed
200
result =  100
stop : 3377699720880596  is treated  0.27374760470845877 % completed
200
result =  100
stop : 3377699720880597  is treated  0.3011223651793047 % completed
200
result =  100
stop : 3377699720880598  is treated  0.328

200
result =  100
stop : 3377699720881135  is treated  2.6553517656720502 % completed
200
result =  100
stop : 3377699720881136  is treated  2.682726526142896 % completed
200
result =  100
stop : 3377699720881137  is treated  2.7101012866137424 % completed
200
result =  100
stop : 3377699720881138  is treated  2.7374760470845882 % completed
200
result =  100
stop : 3377699720881139  is treated  2.764850807555434 % completed
200
result =  100
stop : 3377699720881140  is treated  2.7922255680262795 % completed
200
result =  100
stop : 3377699720881143  is treated  2.8196003284971254 % completed
200
result =  100
stop : 3377699720881144  is treated  2.8469750889679712 % completed
200
result =  100
stop : 3377699720881146  is treated  2.8743498494388176 % completed
200
result =  36
stop : 3377699720881147  is treated  2.9017246099096634 % completed
200
result =  100
stop : 3377699720881148  is treated  2.9290993703805093 % completed
200
result =  100
stop : 3377699720881149  is treated  2.

200
result =  100
stop : 3377699720881319  is treated  5.3107035313441004 % completed
200
result =  100
stop : 3377699720881320  is treated  5.338078291814947 % completed
200
result =  100
stop : 3377699720881321  is treated  5.365453052285792 % completed
200
result =  100
stop : 3377699720881323  is treated  5.3928278127566385 % completed
200
result =  100
stop : 3377699720881324  is treated  5.420202573227485 % completed
200
result =  100
stop : 3377699720881327  is treated  5.44757733369833 % completed
200
result =  100
stop : 3377699720881328  is treated  5.4749520941691765 % completed
200
result =  100
stop : 3377699720881329  is treated  5.502326854640022 % completed
200
result =  100
stop : 3377699720881330  is treated  5.529701615110868 % completed
200
result =  100
stop : 3377699720881331  is treated  5.557076375581714 % completed
200
result =  100
stop : 3377699720881332  is treated  5.584451136052559 % completed
200
result =  100
stop : 3377699720881334  is treated  5.611825

200
result =  100
stop : 3377699720881549  is treated  7.966055297016152 % completed
200
result =  100
stop : 3377699720881550  is treated  7.993430057486997 % completed
200
result =  100
stop : 3377699720881554  is treated  8.020804817957842 % completed
200
result =  100
stop : 3377699720881555  is treated  8.04817957842869 % completed
200
result =  99
stop : 3377699720881565  is treated  8.075554338899535 % completed
200
result =  100
stop : 3377699720881566  is treated  8.10292909937038 % completed
200
result =  100
stop : 3377699720881567  is treated  8.130303859841225 % completed
200
result =  100
stop : 3377699720881576  is treated  8.157678620312073 % completed
200
result =  100
stop : 3377699720881577  is treated  8.185053380782918 % completed
200
result =  100
stop : 3377699720881619  is treated  8.212428141253763 % completed
200
result =  100
stop : 3377699720881620  is treated  8.23980290172461 % completed
200
result =  100
stop : 3377699720881633  is treated  8.267177662195

200
result =  100
stop : 3377699720882036  is treated  10.621407062688201 % completed
200
result =  100
stop : 3377699720882038  is treated  10.648781823159046 % completed
200
result =  100
stop : 3377699720882039  is treated  10.676156583629894 % completed
200
result =  100
stop : 3377699720882040  is treated  10.703531344100739 % completed
200
result =  100
stop : 3377699720882041  is treated  10.730906104571584 % completed
200
result =  100
stop : 3377699720882042  is treated  10.758280865042432 % completed
200
result =  100
stop : 3377699720882043  is treated  10.785655625513277 % completed
200
result =  100
stop : 3377699720882044  is treated  10.813030385984122 % completed
200
result =  100
stop : 3377699720882046  is treated  10.84040514645497 % completed
200
result =  100
stop : 3377699720882172  is treated  10.867779906925815 % completed
200
result =  100
stop : 3377699720882173  is treated  10.89515466739666 % completed
200
result =  100
stop : 3377699720882174  is treated  1

...

200
result =  100
stop : 3377704015496727  is treated  90.09033670955378 % completed
200
result =  100
stop : 3377704015496728  is treated  90.11771147002464 % completed
200
result =  100
stop : 3377704015496729  is treated  90.14508623049548 % completed
200
result =  100
stop : 3377704015496730  is treated  90.17246099096633 % completed
200
result =  100
stop : 3377704015496731  is treated  90.19983575143718 % completed
200
result =  100
stop : 3377704015496732  is treated  90.22721051190801 % completed
200
result =  100
stop : 3377704015496733  is treated  90.25458527237886 % completed
200
result =  100
stop : 3377704015496734  is treated  90.28196003284972 % completed
200
result =  100
stop : 3377704015496735  is treated  90.30933479332056 % completed
200
result =  100
stop : 3377704015496736  is treated  90.33670955379141 % completed
200
result =  100
stop : 3377704015496737  is treated  90.36408431426224 % completed
200
result =  100
stop : 3377704015496738  is treated  90.3914590

200
result =  100
stop : 3377704015496830  is treated  92.74568847522585 % completed
200
result =  100
stop : 3377704015496831  is treated  92.77306323569668 % completed
200
result =  100
stop : 3377704015496832  is treated  92.80043799616753 % completed
200
result =  100
stop : 3377704015496833  is treated  92.82781275663838 % completed
200
result =  100
stop : 3377704015496834  is treated  92.85518751710923 % completed
200
result =  100
stop : 3377704015496835  is treated  92.88256227758008 % completed
200
result =  60
stop : 3377704015496836  is treated  92.90993703805091 % completed
200
result =  60
stop : 3377704015496837  is treated  92.93731179852176 % completed
200
result =  100
stop : 3377704015496838  is treated  92.96468655899261 % completed
200
result =  100
stop : 3377704015496839  is treated  92.99206131946346 % completed
200
result =  100
stop : 3377704015496840  is treated  93.01943607993431 % completed
200
result =  100
stop : 3377704015496841  is treated  93.046810840

200
result =  55
stop : 3377704015496928  is treated  95.42841500136873 % completed
200
result =  65
stop : 3377704015496929  is treated  95.45578976183958 % completed
200
result =  55
stop : 3377704015496930  is treated  95.48316452231043 % completed
200
result =  55
stop : 3377704015496931  is treated  95.51053928278127 % completed
200
result =  72
stop : 3377704015496932  is treated  95.53791404325213 % completed
200
result =  72
stop : 3377704015496933  is treated  95.56528880372296 % completed
200
result =  72
stop : 3377704015496934  is treated  95.59266356419381 % completed
200
result =  72
stop : 3377704015496935  is treated  95.62003832466466 % completed
200
result =  72
stop : 3377704015496936  is treated  95.6474130851355 % completed
200
result =  72
stop : 3377704015496937  is treated  95.67478784560635 % completed
200
result =  72
stop : 3377704015496938  is treated  95.7021626060772 % completed
200
result =  72
stop : 3377704015496939  is treated  95.72953736654804 % comp

200
result =  100
stop : 3377704015497039  is treated  98.11114152751163 % completed
200
result =  100
stop : 3377704015497040  is treated  98.13851628798248 % completed
200
result =  100
stop : 3377704015497041  is treated  98.16589104845332 % completed
200
result =  100
stop : 3377704015497042  is treated  98.19326580892417 % completed
200
result =  100
stop : 3377704015497043  is treated  98.22064056939502 % completed
200
result =  100
stop : 3377704015497044  is treated  98.24801532986586 % completed
200
result =  100
stop : 3377704015497045  is treated  98.27539009033671 % completed
200
result =  100
stop : 3377704015497046  is treated  98.30276485080756 % completed
200
result =  100
stop : 3377704015497047  is treated  98.3301396112784 % completed
200
result =  100
stop : 3377704015497048  is treated  98.35751437174925 % completed
200
result =  100
stop : 3377704015497049  is treated  98.38488913222008 % completed
200
result =  100
stop : 3377704015497050  is treated  98.41226389

{'3377699720880573': [(datetime.datetime(2020, 2, 3, 6, 3, 41), 41.0),
  (datetime.datetime(2020, 2, 3, 6, 34, 3), 63.0),
  (datetime.datetime(2020, 2, 3, 7, 21, 27), 27.0),
  (datetime.datetime(2020, 2, 3, 8, 4, 35), 95.0),
  (datetime.datetime(2020, 2, 3, 9, 23, 5), 65.0),
  (datetime.datetime(2020, 2, 3, 9, 42, 36), 36.0),
  (datetime.datetime(2020, 2, 3, 10, 5, 26), 26.0),
  (datetime.datetime(2020, 2, 3, 11, 35), 0.0),
  (datetime.datetime(2020, 2, 3, 12, 5), 0.0),
  (datetime.datetime(2020, 2, 3, 12, 35), 0.0),
  (datetime.datetime(2020, 2, 3, 13, 5), 0.0),
  (datetime.datetime(2020, 2, 3, 13, 35), 0.0),
  (datetime.datetime(2020, 2, 3, 14, 5), 0.0),
  (datetime.datetime(2020, 2, 3, 14, 35), 0.0),
  (datetime.datetime(2020, 2, 3, 15, 5), 0.0),
  (datetime.datetime(2020, 2, 3, 15, 32), 0.0),
  (datetime.datetime(2020, 2, 3, 15, 59), 0.0),
  (datetime.datetime(2020, 2, 3, 16, 22), 0.0),
  (datetime.datetime(2020, 2, 3, 16, 43), 0.0),
  (datetime.datetime(2020, 2, 3, 17, 6), 0.0),
 

In [11]:
for key_id in result.keys():
    df=pd.DataFrame(result[key_id],columns=['time','retard'])
    df["latitude"] = float(stops[stops["stop_id"]==key_id]["stop_lat"])
    df["logtitude"] = float(stops[stops["stop_id"]==key_id]["stop_lon"])
    df["id"]=key_id
    df.to_csv("~/retard.csv",index=False,mode='a',header = False)